## MLOps - HW1: Data Versioning and Differential Privacy
### Luke Schwenke & Aaron Chan
### October 23, 2023

In [13]:
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn.model_selection import train_test_split
#os.getcwd()

In [11]:
v1 = pd.read_csv('./data/athletes.csv')
v1.head(5)

,athlete_id,name,region,team,affiliate,gender,age,height,weight,fran,...,snatch,deadlift,backsq,pullups,eat,train,background,experience,schedule,howlong
0,2554.0,Pj Ablang,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,NaN,...,NaN,400.0,305.0,NaN,NaN,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|I r...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
1,3517.0,Derek Abdella,NaN,NaN,NaN,Male,42.0,70.0,190.0,NaN,...,NaN,NaN,NaN,NaN,NaN,I have a coach who determines my programming|I...,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5164.0,Abo Brandon,Southern California,LAX CrossFit,LAX CrossFit,Male,40.0,67.0,NaN,211.0,...,200.0,375.0,325.0,25.0,I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|,4+ years|
4,5286.0,Bryce Abbey,NaN,NaN,NaN,Male,32.0,65.0,149.0,206.0,...,150.0,NaN,325.0,50.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I inc...,I played college sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|I strictly s...,1-2 years|


### Data Cleaning

In [16]:
# Remove not relevant columns
v2 = v1.dropna(subset=['region','age','weight','height','howlong','gender','eat', 
                           'train','background','experience','schedule','howlong',
                           'deadlift','candj','snatch','backsq','experience',
                           'background','schedule','howlong'])

v2 = v2.drop(columns=['affiliate','team','name','athlete_id','fran','helen','grace',
                          'filthy50','fgonebad','run400','run5k','pullups','train'])

# Remove Outliers
v2 = v2[v2['weight'] < 1500]
v2 = v2[v2['gender'] != '--']
v2 = v2[v2['age'] >= 18]
v2 = v2[(v2['height'] < 96) & (v2['height'] > 48)]

In [18]:

v2 = v2[(v2['deadlift'] > 0) & (v2['deadlift'] <= 1105)|((v2['gender'] == 'Female') \
                 & (v2['deadlift'] <= 636))]
v2 = v2[(v2['candj'] > 0) & (v2['candj'] <= 395)]
v2 = v2[(v2['snatch'] > 0) & (v2['snatch'] <= 496)]
v2 = v2[(v2['backsq'] > 0) & (v2['backsq'] <= 1069)]

# Clean Survey Data

decline_dict = {'Decline to answer|': np.nan}
v2 = v2.replace(decline_dict)
v2= v2.dropna(subset=['background','experience','schedule','howlong','eat'])

### Response Variable

In [21]:
# must create total_lift variable
v1['total_lift'] = v1['candj'] + v1['snatch'] + v1['deadlift'] + v1['backsq']
v2['total_lift'] = v2['candj'] + v2['snatch'] + v2['deadlift'] + v2['backsq']

### Split and Model

In [23]:
x_v1 = v1.drop('total_lift', axis=1)
y_v1 = v1['total_lift']

x_v2 = v2.drop('total_lift', axis=1)
y_v2 = v2['total_lift']

In [34]:
variables_for_model = ['age','height', 'weight'] # do we need all of them?

x_v1 = x_v1[variables_for_model]
x_v2 = x_v2[variables_for_model]

In [35]:
# v1 split
v1_x_train, v1_x_test, v1_y_train, v1_y_test = train_test_split(x_v1, y_v1, test_size=0.2, random_state=42)

# v2 split
v2_x_train, v2_x_test, v2_y_train, v2_y_test = train_test_split(x_v2, y_v2, test_size=0.2, random_state=42)


In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [37]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
#rf_classifier.fit(v1_x_train, v1_y_train)